# Elo for csgo teams

In [2]:
import pandas as pd
import math

In [3]:
match_history  = pd.read_csv('data/matches.csv',sep = ';')
match_history.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
match_history.head()

,Date,Team1,Team2,Map,Event,event_type,event_tier
0,22/12/19,Vitality (16),MOUZ (9),nuke Nuke,EPICENTER 2019,BigEvents,S
1,22/12/19,Vitality (16),MOUZ (12),mrg Mirage,EPICENTER 2019,BigEvents,S
2,22/12/19,MOUZ (19),Vitality (16),inf Inferno,EPICENTER 2019,BigEvents,S
3,22/12/19,MOUZ (16),Evil Geniuses (14),ovp Overpass,EPICENTER 2019,BigEvents,S
4,22/12/19,MOUZ (16),Evil Geniuses (12),trn Train,EPICENTER 2019,BigEvents,S


## Clean data

In [5]:
match_history.rename(columns={'Date':'date', 'Team1':"team1", 'Team2':"team2", 'Map':"map", 'Event':"event"},inplace=True)
match_history[['team1','s1']] = match_history['team1'].str.split('(',1,expand=True)
match_history[['team2','s2']] = match_history['team2'].str.split('(',1,expand=True)
match_history['team1'] = match_history['team1'].str.strip()
match_history['team2'] = match_history['team2'].str.strip()
match_history['s1'] = match_history['s1'].str[:-1]
match_history['s2'] = match_history['s2'].str[:-1]
match_history['map'] = match_history['map'].str[4:]

### Removing duplicates

In [6]:
match_history.drop_duplicates(subset=match_history.columns.difference(['event_type']),keep='last',inplace=True)

## Change type

### Score

In [7]:
match_history['s1'] = pd.to_numeric(match_history['s1'])
match_history['s2'] = pd.to_numeric(match_history['s2'])

### date

In [8]:
match_history['date'] = pd.to_datetime(match_history['date'])


## Adding information for the table

## Winner

In [9]:
match_history['winner'] = (match_history['s1'] - match_history['s2']>0)

## Elo collumn

In [10]:
match_history['elo1'] = 1500    
match_history['elo2'] = 1500

## Index

In [11]:
match_history.sort_values(by=["date"],inplace=True)
match_history.reset_index(drop=True,inplace=True)
match_history['index1'] = match_history.index

In [12]:
match_history.head(20)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
0,2019-01-03,MIBR,Renegades,Dust2,IEM Katowice 2019,Majors,S,16,6,True,1500,1500,0
1,2019-01-03,Renegades,MIBR,Train,IEM Katowice 2019,Majors,S,12,16,False,1500,1500,1
2,2019-01-03,Ninjas in Pyjamas,Astralis,Dust2,IEM Katowice 2019,Majors,S,14,16,False,1500,1500,2
3,2019-01-03,Astralis,Ninjas in Pyjamas,Mirage,IEM Katowice 2019,Majors,S,16,2,True,1500,1500,3
4,2019-01-04,BIG,paiN,Cache,StarSeries i-League Season 7,BigEvents,S,16,6,True,1500,1500,4
5,2019-01-04,ViCi,ENCE,Inferno,StarSeries i-League Season 7,BigEvents,S,6,16,False,1500,1500,5
6,2019-01-04,MIBR,NRG,Mirage,StarSeries i-League Season 7,BigEvents,S,7,16,False,1500,1500,6
7,2019-01-04,MIBR,NRG,Inferno,StarSeries i-League Season 7,BigEvents,S,5,16,False,1500,1500,7
8,2019-01-04,Spirit,TYLOO,Overpass,StarSeries i-League Season 7,BigEvents,S,16,7,True,1500,1500,8
9,2019-01-04,TYLOO,Spirit,Inferno,StarSeries i-League Season 7,BigEvents,S,6,16,False,1500,1500,9


# Applying elo function

In [29]:
def eloretriver(team,index):
    #retrive the last elo point from a data frame.
    last_elo = match_history[(match_history['index1'] < index) & ((match_history['team1'] == team)|(match_history['team2'] == team))].tail(1)
    if len(last_elo) == 0:
        return None
    elif last_elo['team1'].iloc[0] == team:
        print(last_elo['index1'].iloc[0])
        return last_elo['elo1'].iloc[0]
    elif last_elo['team2'].iloc[0] == team:
        print(last_elo['index1'].iloc[0])
        return last_elo['elo2'].iloc[0]

def Probwin(rating1,rating2):
    # Function to calculate the Probability
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))

def EloRatingcalculator(team1,team2,index1,winner,k,):
    Ra = eloretriver(team1,index1)
    Rb = eloretriver(team2,index1)
    # If team has no rating the starting rating is 1500(?change)
    if Ra is None:
        Ra = 1500
    if Rb is None:
        Rb = 1500
    # Function to calculate Elo rating
    # K is a constant.
    # winner determines whether Player A wins or Player B
    # To calculate the Winning
    # Probability of Player B
    print(Ra)
    print(Rb)
    Pb = Probwin(Ra, Rb)
    # To calculate the Winning
    # Probability of Player A
    Pa = Probwin(Rb, Ra)
      # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (winner == True) :
        elo1 = Ra + k * (1 - Pa)
        elo2 = Rb + k * (0 - Pb)
    # Case -2 When Player B wins
    # Updating the Elo Ratings
    else :
        elo1 = Ra + k * (0 - Pa)
        elo2 = Rb + k * (1 - Pb)
    return pd.Series([round(elo1),round(elo2)])
    

K-factor  	Used for players with ratings ...
k=40 for a player new to the rating list until the completion of events with a total of 30 games, and for all players until their 18th birthday, as long as their rating remains under 2300.
k=20 	for players who have always been rated under 2400.
k=10for players with any published rating of at least 2400 and at least 30 games played in previous events. Thereafter it remains permanently at 10.

Score after mathces,series or lan torunaments(greater difference for tournaments?? Performance Rating by tournament?)

elo starting at what?

elo for players?

In [23]:
match_history[['elo1','elo2']] = match_history.apply(lambda x : EloRatingcalculator(x['team1'],x['team2'],x['index1'],x['winner'],30),axis=1)
#match_history['elo2'] = match_history.apply(lambda x : EloRatingcalculator(x['team2'],x['team1'],x['index1'],~(x['winner']),30),axis=1)

None
None
0
1485
0
1515
None
None
2
1515
2
1485
None
None
None
None
1
1515
None
6
1485
6
1515
None
None
8
1485
8
1515
9
1485
9
1515
None
3
1485
None
None
11
1515
11
1485
4
1515
4
1485
None
None
15
1485
15
1515
13
1515
13
1485
12
1485
12
1515
5
1485
5
1515
None
1
1485
20
1515
20
1485
21
1485
21
1515
None
14
1515
23
1515
23
1485
24
1485
24
1515
18
1515
7
1485
26
1515
26
1485
19
1485
None
28
1485
28
1515
29
1515
29
1485
None
None
31
1515
31
1485
22
1485
7
1515
33
1515
33
1485
22
1515
None
35
1515
35
1485
32
1485
32
1515
34
1485
None
38
1515
38
1485
None
None
40
1485
40
1515
36
1485
39
1485
42
1485
42
1515
43
1515
43
1485
36
1515
41
1515
17
1515
None
45
1485
45
1515
46
1485
46
1515
48
1515
48
1485
3
1515
44
1515
None
41
1485
51
1515
51
1485
52
1515
52
1485
19
1515
18
1485
37
1515
None
54
1515
54
1485
53
1485
50
1515
50
1485
53
1515
27
1485
56
1485
56
1515
10
1515
16
1515
49
1485
60
1515
59
1515
58
1515
57
1515
None
47
1515
47
1485
17
1485
59
1485
60
1485
66
1515
66
1485
67
1515
67
1485
64


In [26]:
match_history.query('team1 == "MIBR" | team2 == "MIBR"').head(50)

,date,team1,team2,map,event,event_type,event_tier,s1,s2,winner,elo1,elo2,index1
0,2019-01-03,MIBR,Renegades,Dust2,IEM Katowice 2019,Majors,S,16,6,True,1515,1485,0
1,2019-01-03,Renegades,MIBR,Train,IEM Katowice 2019,Majors,S,12,16,False,1471,1529,1
6,2019-01-04,MIBR,NRG,Mirage,StarSeries i-League Season 7,BigEvents,S,7,16,False,1499,1516,6
7,2019-01-04,MIBR,NRG,Inferno,StarSeries i-League Season 7,BigEvents,S,5,16,False,1471,1529,7
26,2019-01-05,fnatic,MIBR,Overpass,IEM Sydney 2019,BigEvents,S,9,16,False,1499,1501,26
27,2019-01-05,MIBR,fnatic,Nuke,IEM Sydney 2019,BigEvents,S,16,4,True,1529,1471,27
83,2019-02-03,MIBR,Astralis,Inferno,IEM Katowice 2019,Majors,S,7,16,False,1500,1530,83
84,2019-02-03,Astralis,MIBR,Overpass,IEM Katowice 2019,Majors,S,16,14,True,1529,1471,84
86,2019-02-04,MIBR,North,Inferno,StarSeries i-League Season 7,BigEvents,S,8,16,False,1470,1500,86
97,2019-02-04,North,MIBR,Overpass,StarSeries i-League Season 7,BigEvents,S,16,10,True,1529,1471,97


In [30]:
EloRatingcalculator('MIBR','NRG',6,False,30)

1
1529
None
1529
1500


0    1513
1    1516
dtype: int64